In [ ]:
%pylab notebook
# check if a windows machine, it needs special attention
# this extra step will bypass an error from mpl_toolkits.basemap
import os
if os.name == 'nt':
    os.environ["PROJ_LIB"] = os.path.join(os.environ["CONDA_PREFIX"], "Library", "share")
    os.environ["GDAL_DATA"] = os.path.join(os.environ["CONDA_PREFIX"], "Library", "share", "gdal")
import matplotlib.pyplot as plt
from mpl_toolkits.axes_grid1 import make_axes_locatable
from netCDF4 import Dataset, num2date
import numpy as np
from osgeo import gdal, osr   # noqa
import pandas as pd
from pathlib import Path
import re
from cetbtools.ease2conv import Ease2Transform
from mpl_toolkits.basemap import Basemap
import glob
from pathlib import Path

# Read a GRD MOD pkl file with geolocation

In [ ]:
dataDir = Path(Path.home(), 'nsidc0630_v1') # Mariah's PC or Mary Jo's Mac
scriptDir = Path(Path.home(), 'ipynb_melt_onset', 'scripts')

modsDir = "%s/MODs" % dataDir
modsDir

In [ ]:
%cd $scriptDir

In [ ]:
from CETB_write_functions import write_MOD_df_to_geotiff

In [ ]:
os.chdir(modsDir)
gpd = "EASE2_N3.125km"
modFiles = sort(glob.glob("%s-WesternCA*MOD.pkl" % gpd))
modFiles

In [ ]:
fidx = 0
MOD_df = pd.read_pickle(modFiles[fidx])

In [ ]:
MOD_df

In [ ]:
%pwd

In [ ]:
outbasename = 'test.bathurst_range'
out = write_MOD_df_to_geotiff(MOD_df, gpd, outbasename, verbose=False)

In [ ]:
out


In [ ]:
files.append('newname.tif')
files

# Initialize a grid object for the transformations between lat/lon, row/col and x/y

This will depend on whether data are GRD (25 km) or SIR (6.25 or 3.125)

In [ ]:
from cetbtools import ease2conv
grid = ease2conv.Ease2Transform(gridname=gpd)


# Function that takes a column of the original MOD df and reshapes it to a 2D array

In [ ]:
def write_MOD_df_column_to_geotiff(df, column, outfilename, grid, dtype='int16', verbose=False):

    nrows = int(df.iloc[-1].row - df.iloc[0].row + 1)
    ncols = int(df.iloc[-1].column - df.iloc[0].column + 1)
    
    # Coerce array data to requested dtype
    data = np.array(df[column].values.data).reshape(nrows, ncols)
    data = data.astype(dtype)
    if verbose:
        print('data dtype = %s, nrows = %d, ncols = %d' % 
              (data.dtype, nrows, ncols),
             file=sys.stderr,
             flush=True)
    
    if ("float32" == data.dtype):                                                                    
        gdal_data_type = gdal.GDT_Float32                                                       
    elif ("int8" == data.dtype):                                                                  
        gdal_data_type = gdal.GDT_Byte                                                          
    elif ("int16" == data.dtype):                                                                  
        gdal_data_type = gdal.GDT_Int16                                                         
    else:                                                                                       
        print("%s : Unrecognized type %s " %                                                 
              (my_name, str(data.dtype)),                                             
              file=sys.stderr,                                                                  
              flush=True)                                                                       
        raise ValueError                                                                        

    # Initialize the output driver
    # Documentation for raster GTiff driver here: https://gdal.org/drivers/raster/gtiff.html#raster-gtiff
    driver = gdal.GetDriverByName("GTiff")                                                      
                                                                                                
    # use this to control block sizes:
    # dest_ds_options = ['COMPRESS=LZW', 'TILED=YES', 'BLOCKXSIZE=256', 'BLOCKYSIZE=256']  
    dest_ds_options = ['COMPRESS=LZW']
    dest_ds = driver.Create(outfilename, ncols, nrows, 1, gdal_data_type, dest_ds_options)
    
    # Initialize the projection information                                                     
    # The crs.proj4text attribute can also be used here,                                        
    # but the srid attribute provides more complete PROJCRS metadata                            
    proj = osr.SpatialReference()                                                                                                                   
    proj.SetFromUserInput(grid.epsg)                                                             
    dest_ds.SetProjection(proj.ExportToWkt())
    
    # Initialize the grid information (extent and scale)                                        
    # Thanks to web page at:                                                                    
    # http://geoexamples.blogspot.com/2012/01/                                                  
    # creating-files-in-ogr-and-gdal-with.html                                                  
    # The geotransform defines the relation between the                                         
    # raster coordinates x, y and the                                                           
    # geographic coordinates, using the following definition:                                   
    # Xgeo = geotransform[0] + Xpixel*geotransform[1] + Yline*geotransform[2]                   
    # Ygeo = geotransform[3] + Xpixel*geotransform[4] + Yline*geotransform[5]                   
    # The first and fourth parameters define the origin of the upper left pixel                 
    # The second and sixth parameters define the pixels size.                                   
    # The third and fifth parameters define the rotation of the raster.                         
    # Values are meters                                                                         
    # The UL information is the center of the UL corner pixel in projected
    # coordinates
    ULrow, ULcol = grid.geographic_to_grid(df.iloc[0].latitude, df.iloc[0].longitude)
    print(ULrow, ULcol)
    ULrow = int(ULrow + 0.5)
    ULcol = int(ULcol + 0.5)
    
    map_ULx, map_ULy = grid.grid_to_map(ULrow, ULcol)
    print("UL: ", ULrow, ULcol, df.iloc[0].latitude, df.iloc[0].longitude, map_ULx, map_ULy)
    
    LRrow, LRcol = grid.geographic_to_grid(df.iloc[-1].latitude, df.iloc[-1].longitude)
    LRrow = int(LRrow + 0.5)
    LRcol = int(LRcol + 0.5)
    map_LRx, map_LRy = grid.grid_to_map(LRrow, LRcol)
    print("LR: ", LRrow, LRcol, df.iloc[-1].latitude, df.iloc[-1].longitude, map_LRx, map_LRy)
    
    # Get the projection scales by dividing the projected extents from UL and LR pixels
    # by row/col dimensions
    scale_x = (map_LRx - map_ULx) / double(ncols - 1)
    scale_y = -1. * (map_ULy - map_LRy) / double(nrows - 1)
    print('scale x, y = %f, %f' % (scale_x, scale_y))

    cornerULx = map_ULx - (scale_x / 2.)
    cornerULy = map_ULy - (scale_y / 2.)
    print("cornerUL: ", cornerULx, cornerULy)
    geotransform = (cornerULx, scale_x, 0., cornerULy, 0., scale_y)                               
    dest_ds.SetGeoTransform(geotransform)                                                       
    dest_ds.GetRasterBand(1).WriteArray(data) 
                                                     
    dest_ds = None

    if verbose:                                                                                 
        print("\n%s geotiff image saved to: %s" %                                          
              (str(column), outfilename),                                                 
              file=sys.stderr,                                                                  
              flush=True)  

In [ ]:
column = 1995
outfilename = 'test.tif'
write_df_column_to_geotiff(MOD_df, column, outfilename, grid, verbose=True)

In [ ]:
%pwd


In [ ]:
for column in MOD_df:
    print('next array is %s' % column)
    data = np.array(MOD_df[column].values.data).reshape(nrows, ncols)
    print(vals.dtype)

In [ ]:
data = np.reshape()